## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta
from scipy.interpolate import interp1d
from scipy.fftpack import fft
from sklearn.preprocessing import MinMaxScaler

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'rotor'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['g1_sensor1_normal',
 'g1_sensor1_type1',
 'g1_sensor1_type2',
 'g1_sensor1_type3',
 'g1_sensor2_normal',
 'g1_sensor2_type1',
 'g1_sensor2_type2',
 'g1_sensor2_type3',
 'g1_sensor3_normal',
 'g1_sensor3_type1',
 'g1_sensor3_type2',
 'g1_sensor3_type3',
 'g1_sensor4_normal',
 'g1_sensor4_type1',
 'g1_sensor4_type2',
 'g1_sensor4_type3',
 'g2_sensor1_normal',
 'g2_sensor1_type1',
 'g2_sensor1_type2',
 'g2_sensor1_type3',
 'g2_sensor2_normal',
 'g2_sensor2_type1',
 'g2_sensor2_type2',
 'g2_sensor2_type3',
 'g2_sensor3_normal',
 'g2_sensor3_type1',
 'g2_sensor3_type2',
 'g2_sensor3_type3',
 'g2_sensor4_normal',
 'g2_sensor4_type1',
 'g2_sensor4_type2',
 'g2_sensor4_type3']

## TAG Name format 변환 

* 위의 과정에서 rotor dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

* g1 Tag Name 사용하여 예제 진행 

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[:16]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'g1_sensor1_normal','g1_sensor1_type1','g1_sensor1_type2','g1_sensor1_type3','g1_sensor2_normal','g1_sensor2_type1','g1_sensor2_type2','g1_sensor2_type3','g1_sensor3_normal','g1_sensor3_type1','g1_sensor3_type2','g1_sensor3_type3','g1_sensor4_normal','g1_sensor4_type1','g1_sensor4_type2','g1_sensor4_type3'
16


## Rotor Dataset 로드

* 데이터 로드시 전체 데이터 셋을 각각 Load

* label 설명
    * normal : 정상
    * type1 : Disk 2에 회전 불균형 (270도 위치에 볼트, 너트 부착)
    * type2 : Support 4에 지지 불균형
    * type3 : Type 1 + Type 2

In [6]:
# 데이터 시간 로드 함수
def time_data_load(table, name, start_time, end_time, timeformat):
    
    target = 'time'
    
    # 데이터 로드 
    df = pd.read_csv(f"http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?target={target}&table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}")
    
    # resample을 위해 임의의 value 컬럼 생성
    df['value'] = 0
    
    # resample 진행
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    df = df.resample('1s').mean()
    
    # 결측값 제거
    df = df.dropna()
    
    # 임의의 value 컬럼 제거
    df = df.drop(['value'], axis=1)
    
    return df

In [7]:
# 컬럼 이름에서 레이블 추출 함수
def get_label(column_name):
    
    # 레이블 매핑
    label_mapping = {
        'normal': 0,
        'type1': 1,
        'type2': 2,
        'type3': 3
    }
    
    column_name = str(column_name)
    for key in label_mapping.keys():
        if key in column_name:
            return label_mapping[key]
    return None

In [8]:
# 데이터 로드 함수
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time 설정
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # 빈 데이터 프레임 생성
    data_result = pd.DataFrame()

    # 각 tag name 별 데이터 보간

    for i in range(len(df.columns[1:])):
        
        # 시간 설정
        start = pd.to_datetime(unquote(start_time))
        end = pd.to_datetime(unquote(end_time))
        
        data_ = df.iloc[ : , [0] + list(range(i+1, i+2))].dropna()
        
        # 보간할 새로운 시간 구간 생성 (1초마다 1000개의 포인트)
        # 이 경우, 원본 데이터는 1ms 간격으로 측정되었으므로 1초 간격으로 1000개의 포인트를 생성
        # 0에서 140 구간을 생성 -> 140,000
        new_time_range = pd.date_range(start=start, end=end, freq='1ms')[:-1]
        new_time_range_ = pd.date_range(start=start, end=end, freq='1s')[:-1]

        # 선형 보간을 사용하여 데이터 보간
        # datetime을 숫자형으로 변환 (epoch time in seconds)
        time_numeric = pd.to_numeric(data_['TIME'])
        new_time_numeric = pd.to_numeric(new_time_range)

        value = data_[data_.columns[1:].values]

        # 선형 보간 객체 생성
        interpolator = interp1d(time_numeric, value.values.reshape(-1), kind='linear', fill_value='extrapolate')
        interpolated_values = interpolator(new_time_numeric)
        interpolated_values = np.clip(interpolated_values, min(value.values), max(value.values))

        # 데이터 프레임 생성
        data_remake = pd.DataFrame(interpolated_values.reshape(-1,1000))
        data_remake['time'] = new_time_range_
        data_remake['sensor'] = f'{data_.columns[1:].item()}'

        # 이동할 컬럼과 새로운 순서 지정
        cols = data_remake.columns.tolist()
        cols.insert(0, cols.pop(cols.index('time')))
        cols.insert(1, cols.pop(cols.index('sensor')))
        data_remake = data_remake[cols]

        # 빈 데이터 프레임에 추가
        data_result = pd.concat([data_result, data_remake], ignore_index=True)
        
    # 시간순 정렬
    data_result = data_result.sort_values(by='time').reset_index(drop=True)
    
    # label 설정 
    # 각 컬럼에 대해 레이블을 적용하여 새로운 시리즈 생성
    labels = pd.Series(data_result['sensor']).map(get_label)

    # 데이터 프레임에 label 추가
    data_result['label'] = labels.values

    # 시간, 센서 이름 컬럼 제거
    data_result = data_result.iloc[:,2:]
        
    return data_result

In [9]:
# 시간 변환 함수
# 시간 추가하려면 해당 과정 필요
def add_time(time_df, start_time, batch_size):
    
    # 몇개의 데이터를 로드해야 되는지 계산
    time = int(batch_size / 16)
    
    # 현재 시간의 인덱스 번호를 확인
    # 없는 경우는 맨처음 시간이 없는 경우이기 때문에 맨처음 인덱스로 지정함 
    try:
        index_now = time_df.index.get_loc(start_time)
    except KeyError:
        index_now = 0
    
    # 현재 시간 기준 배치 데이터의 마지막 시간 설정 
    end_time_ = str(time_df.index[index_now + time])
    
    # 다음 시작 시간의 인덱스 번호 설정
    index_next = index_now + time
    
    # 다음 시작 시간 설정
    next_start_time_ = str(time_df.index[index_next])
    
    # URL 인코딩
    start_time_ = quote(start_time)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_, index_next

## 데이터 전처리
   * hanning window, FFT, MinMaxScaling 적용

## hanning window 함수 설정

In [10]:
# hanning window 함수 설정 
def set_hanning_window(sample_rate, df):
    
    # Hanning 윈도우 생성
    hanning_window = np.hanning(sample_rate)

    # 각 행에 Hanning 윈도우 적용
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

## FFT 함수 설정

In [11]:
# FFT 변환 함수
def change_fft(sample_rate, df):
    # 신호의 총 샘플 수
    N = sample_rate
    
    # 각 행에 대해 FFT 적용
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    for i in range(df.shape[0]):
        # 각 행의 FFT 계산
        yf = fft(df.iloc[i].values)
        
        # FFT 결과의 절댓값을 계산하고 정규화 (유의미한 부분만)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # FFT 결과를 데이터 프레임으로 변환
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

## 학습 모델 설정 

* 1D ResNet 모델 사용

In [12]:
# 1D ResNet 모델 사용 
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [13]:
# 모델 설정 파라미터
# 학습률 
learning_rate = 0.01

# 모델 초기화
model = ResNet1D(ResidualBlock, [2, 2, 2, 2], num_classes=4).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## 모델 학습 설정

* 학습 중 validation 데이터 기준 F1 score 값이 제일 높은 모델을 저장

In [14]:
# 모델 학습 함수 설정
def train(table, name, timeformat, model, start_time_train, end_time_train, start_time_valid, end_time_valid, batch_size, epochs, scaler, time_df_train, time_df_valid, sample_rate):
    
    # 초기 train loss 설정
    train_loss = []
    train_acc = []

    # 베스트 f1 값 초기화
    best_f1= -np.inf

    for epoch in epochs:
        
        # 학습 모드 설정 
        model.train()

        running_loss = 0.0
        total_step = 0
        correct = 0
        total=0

        # 초기 시작 시간 설정
        start_time_ = start_time_train

        # 끝 시간 설정
        end_time_train = str(time_df_train.index[-1])

        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_train:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_, end_time_, next_start_time_, index_next= add_time(time_df_train, start_time_, batch_size)
            
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat)
            
            # hanning window 적용
            data_ = set_hanning_window(sample_rate, data.iloc[:,:-1])
            
            # FFT 적용 
            data_  = change_fft(sample_rate, data_ )
            
            # MinMax scaler 적용 
            data_ = scaler.fit_transform(data_)
            
            # 데이터 프레임 + label 설정
            data_ = pd.DataFrame(data_)
            data_['label'] = data['label'].values
            
            # 데이터 랜덤 셔플
            data_ = data_.sample(frac=1).reset_index(drop=True)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(data_) == batch_size:
                
                # 총 배치수 체크용  
                total_step = total_step + 1
                
                # 데이터를 numpy 배열로 변환
                input_data = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                label = np.array(data_.iloc[:,-1:])

                # 데이터를 Tensor로 변환
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()
                label = torch.tensor(label).to(device).long().squeeze()

                # 옵티마이저 최적화 
                optimizer.zero_grad()
                
                # 모델 입력
                outputs = model(input_data)
                
                # loss 계산
                loss = criterion(outputs, label)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # label 예측 값 설정 
                _,pred = torch.max(outputs, dim=1)
                correct += torch.sum(pred==label).item()
                total += label.size(0)
                
                # 배치 리셋
                data_ = 0
                
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_)
                
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next + (int(batch_size /16)) >= len(time_df_train):
                break
            
        train_acc.append(100 * correct / total)
        train_loss.append(running_loss/total_step)
        print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')

        # Epoch 마다 validation을 진행해서 가장 좋은 성능을 보이는 모델을 저장 
        with torch.no_grad():
            
            model.eval()
            
            # 초기화
            preds_ = []
            targets_ = []
                
            # 초기 시작 시간 설정
            start_time_v = start_time_valid
            
            # 끝 시간 설정
            end_time_valid = str(time_df_valid.index[-1])
            
            # while 문을 통해 데이터 호출 
            while start_time_v < end_time_valid:
                
                # 배치 크기에 따라 데이터 로드 
                start_time_v, end_time_v, next_start_time_v, index_next_v = add_time(time_df_valid, start_time_v, batch_size)
                
                # 데이터 로드 
                data_v = data_load(table, name, start_time_v, end_time_v, timeformat)
                
                # hanning window 적용
                data_ = set_hanning_window(sample_rate, data_v.iloc[:,:-1])
                
                # FFT 적용 
                data_  = change_fft(sample_rate, data_ )
                
                # MinMax scaler 적용 
                data_ = scaler.fit_transform(data_)
                
                # 데이터 프레임 + label 설정
                data_ = pd.DataFrame(data_)
                data_['label'] = data_v['label'].values
                
                # 데이터 랜덤 셔플
                data_ = data_.sample(frac=1).reset_index(drop=True)
                
                # 로드한 데이터가 비어 있을 경우 출력 
                if len(data_) == 0:
                    print("데이터가 없습니다.")
                
                # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
                if len(data_) == batch_size:
                    
                    # 데이터를 numpy 배열로 변환
                    input_data_v = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                    label_v = np.array(data_.iloc[:,-1:])

                    # 데이터를 Tensor로 변환
                    input_data_v = torch.tensor(input_data_v, dtype=torch.float32).to(device).float()
                    label_v = torch.tensor(label_v).to(device).long().squeeze()
                    
                    # 모델 입력
                    outputs_v = model(input_data_v)
                    
                    # label 예측 값 설정 
                    _,pred_v = torch.max(outputs_v, dim=1)
                    target_v = label_v.view_as(pred_v)
      
                    preds_.append(pred_v)
                    targets_.append(target_v)
                    
                    # 배치 리셋
                    data_ = 0
                    
                # 다음 시작 시간 설정    
                start_time_v = unquote(next_start_time_v)
                
                # 마지막 시간을 넘어 가져오는 것을 방지
                if index_next_v + (int(batch_size /16)) >= len(time_df_valid):
                    break
                    
            # 모든 배치에서 수집된 예측과 라벨을 합침
            preds_v = torch.cat(preds_).detach().cpu().numpy()
            targets_v = torch.cat(targets_).detach().cpu().numpy()
            
            f1score = f1_score(targets_v, preds_v,  average='macro')
            if best_f1 < f1score:
                best_f1 = f1score
                # 베스트 모델 저장 
                with open("./result/rotor_1d_ResNet_New_Batch.txt", "a") as text_file:
                    print('epoch=====',epoch, file=text_file)
                    print(classification_report(targets_v, preds_v, digits=4), file=text_file)
                torch.save(model, f'./result/rotor_1d_ResNet_New_Batch.pt') 
            epochs.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")
               
    return model

In [15]:
########################################### 학습 파라미터 설정 ################################################
# tag table 이름 설정
table = 'rotor'
# tag name 설정
name = quote(tags_, safe=":/")
# 학습 시작 시간 설정
start_time_train = '2024-01-01 00:00:00'
# 학습 끝 시간 설정
end_time_train = '2024-01-01 00:01:39'
# 시간 포멧 설정 
timeformat = quote('2006-01-02 15:04:05.000000')
# 배치 사이즈 설정
batch_size = 16
# epoch 설정
epochs = trange(30, desc='training')
# sample rate 설정
sample_rate = 1000
# Min-Max scaler 설정 
scaler = MinMaxScaler()
# 학습 시간 리스트 로드 
time_df_train = time_data_load(table, name, quote(start_time_train), quote(end_time_train), timeformat)

######################################## validation 파라미터 설정 #############################################

# 검증 시작 시간 설정
start_time_valid = '2024-01-01 00:01:39'
# 검증 끝 시간 설정
end_time_valid = '2024-01-01 00:02:00'

# 검증 시간 리스트 로드
time_df_valid = time_data_load(table, name, quote(start_time_valid), quote(end_time_valid), timeformat)

################################################ 학습 진행 ####################################################
train(table, name, timeformat, model, start_time_train, end_time_train, start_time_valid, end_time_valid, batch_size, epochs, scaler, time_df_train, time_df_valid, sample_rate)

training:   0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.1887357015998996, train acc: 49.1071

train loss: 1.0850530114709114, train acc: 53.9541

train loss: 0.9714551280144931, train acc: 66.5179

train loss: 0.8701754094918771, train acc: 76.5306

train loss: 0.7768268527349038, train acc: 83.8648

train loss: 0.698825776279226, train acc: 88.4566

train loss: 0.6311774903788903, train acc: 92.5383

train loss: 0.5699560408342668, train acc: 94.7704

train loss: 0.524442224559013, train acc: 94.0689

train loss: 0.4860002517110991, train acc: 94.8980

train loss: 0.45642082132347245, train acc: 94.1964

train loss: 0.42380084072716445, train acc: 97.5765

train loss: 0.39790750064295344, train acc: 97.5765

train loss: 0.3710970530060219, train acc: 99.1709

train loss: 0.3482742648154377, train acc: 98.9796

train loss: 0.3296967620554657, train acc: 97.8316

train loss: 0.31148497769011296, train acc: 99.3622

train loss: 0.2973461289206616, train acc: 98.2781

train loss: 0.28358842209458396, train acc: 98.5969

train lo

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## 모델 테스트

In [20]:
# 베스트 모델 로드
model_ = torch.load(f'./result/rotor_1d_ResNet_New_Batch.pt') 

In [21]:
## 테스트 함수 
## 테스트 함수 
def test(table, name, timeformat, model, start_time_test, end_time_test, batch_size, sample_rate, scaler, time_df_test):

    with torch.no_grad():
        
        model.eval()
        
        # 초기 시작 시간 설정
        start_time_t = start_time_test
        
        # 끝 시간 설정
        end_time_test = str(time_df_test.index[-1])
        
        # label 리스트 설정
        preds_test = []
        target_test = []
        
        # while 문을 통해 데이터 호출 
        while start_time_t < end_time_test:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_t, end_time_t, next_start_time_t, index_next_t = add_time(time_df_test, start_time_t, batch_size)
            
            # 데이터 로드 
            data_v = data_load(table, name, start_time_t, end_time_t, timeformat)
            
            # hanning window 적용
            data_ = set_hanning_window(sample_rate, data_v.iloc[:,:-1])
            
            # FFT 적용 
            data_  = change_fft(sample_rate, data_ )
            
            # MinMax scaler 적용 
            data_ = scaler.fit_transform(data_)
            
            # 데이터 프레임 + label 설정
            data_ = pd.DataFrame(data_)
            data_['label'] = data_v['label'].values
            
            # 데이터 랜덤 셔플
            data_ = data_.sample(frac=1).reset_index(drop=True)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(data_) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_test = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                input_data_label = np.array(data_.iloc[:,-1:])
                
                # 데이터를 Tensor로 변환
                input_data_test = torch.tensor(input_data_test, dtype=torch.float32).to(device).float()
                input_data_label = torch.tensor(input_data_label, dtype=torch.float32).to(device).long()
                
                # DataLoader 생성
                dataset = TensorDataset(input_data_test, input_data_label)
                data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
                
                for batch_input, batch_label in data_loader:

                    # 모델 입력
                    outputs_t = model(batch_input)
                    
                    # 예측 label 확인 
                    _,pred_t = torch.max(outputs_t, dim=1)
                    targets_t = batch_label.view_as(pred_t).to(device)
                    
                    preds_test.append(pred_t)
                    target_test.append(targets_t)
                
                # 배치 리셋
                data_ = []
                
            # 다음 시작 시간 설정    
            start_time_t = unquote(next_start_time_t) 
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next_t + (int(batch_size /16)) >= len(time_df_test):
                break
            
    # 모든 배치에서 수집된 예측과 라벨을 합침
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

    # 결과 데이터 프레임 생성
    
    final_df = pd.DataFrame(target_test, columns=['label'])
    final_df['pred'] = target_test
    
    return final_df

In [22]:
######################################## 테스트 파라미터 설정 #############################################

# 검증 시작 시간 설정
start_time_test = '2024-01-01 00:01:59'
# 검증 끝 시간 설정
end_time_test = '2024-01-01 00:02:20'
# 검증 시간 리스트 로드
time_df_test = time_data_load(table, name, quote(start_time_test), quote(end_time_test), timeformat)

######################################## 테스트 진행 #############################################
final_df = test(table, name, timeformat, model_, start_time_test, end_time_test, batch_size, sample_rate, scaler, time_df_test)

## 모델 성능 확인 

In [23]:
print(classification_report(final_df['label'].values, final_df['pred'].values))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00        80
           2       1.00      1.00      1.00        80
           3       1.00      1.00      1.00        80

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320

